Assignment 1: Camera Calibration
ANTONIO PIO SCIACCHITANO



In [18]:
import cv2
import numpy as np
import os

path_image = "imgProva2/"
lista_img = os.listdir(path_image)
chessboard_size = (6, 9) #dim scacchiera
def find_correspondences(path_image, lista_img, chessboard_size):
    # Coordinate dei punti del quadrato nel mondo reale
    #array NumPy con 3 dimensioni 6 x 9 x 3 e un tipo di dato float a 32-bit
    # L'array è inizializzato con tutti i valori 0
    obj_points = np.zeros((6 * 9, 3), np.float32)
    obj_points[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
    imgpoints = []
    objpoints_list = []  # lista di punti 3D
    # Definisco la lista di punti di interesse per ogni immagine
    corners_list = []  # lista di punti 2D

    # Leggi tutte le immagini nella cartella
    for im_name in lista_img:
        im = cv2.imread(path_image + im_name)
        #la funzione findChessboardCorners rileva gli angoli di una scacchiera in una immagine.
        retval, corners = cv2.findChessboardCorners(im, chessboard_size)
        if retval:
            obj_points = np.squeeze(obj_points)
            objpoints_list.append(obj_points)
            #raffinare le coordinate dei pixel per dati punti 2D.
            corners = np.squeeze(corners)
            imgpoints.append(corners)#stessi elementi con corners_list
            corners_list.append(corners)#stessi elementi con imgpoints

            #la funzione drawChessboardCorners disegna e visualizza gli angoli
            img = cv2.drawChessboardCorners(im, chessboard_size, corners, retval)
            cv2.imshow("im", im)
            cv2.waitKey(0)
    return obj_points, imgpoints, objpoints_list, corners_list, corners, img

In [19]:
obj_points, imgpoints, objpoints_list, corners_list, corners, img = find_correspondences(path_image, lista_img, chessboard_size)

#viene calcolata la matrice di calibrazione K_calibrate, la distorsione dist, e i vettori di rotazione e traslazione per ogni immagine
ret, K_calibrate, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints_list, imgpoints, img.shape[0:2], None, None)
print("matrice K con calibrateCamera\n", K_calibrate)

matrice K con calibrateCamera
 [[1.41459205e+04 0.00000000e+00 3.68579693e+02]
 [0.00000000e+00 9.29432640e+03 6.75578042e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In questa funzione denominata sistema_A_P che prende in input due array numpy: obj_points  che contiene i punti 3D
(x, y, z) corners che contiene i corrispondenti punti 2D (x, y) nella stessa prospettiva.

![punti2D3D](punti2D3D.jpg)


In [20]:
def sistema_A_P(obj_points, corners):

    A_rows = 2 * len(corners)
    #matrice con A_rows(54) righe e 9 colonne, e tutti i suoi elementi sono zero.
    matrice_A = np.zeros((A_rows, 9))
    lista_vettori = []

    #esegue un ciclo for sulla lunghezza degli array corners.
    #All'interno del ciclo, vengono estratti i valori dei punti 3D e 2D dalle rispettive matrici.
    for i in range(len(corners)):
        spazio_X, spazio_Y, _ = obj_points[i] #punti 3D
        pixel_x, pixel_y = corners[i] #punti 2D
        # Vengono creati due vettori che rappresentano
        # la riga i-esima e la riga (i+1)-esima di matrice_A,
        # aggiungendo i valori appropriati a lista_vettori
        lista_vettori.append([spazio_X, spazio_Y, 1, 0, 0, 0, -pixel_x * spazio_X, -pixel_x * spazio_Y, -pixel_x])
        lista_vettori.append([0, 0, 0, spazio_X, spazio_Y, 1, -pixel_y * spazio_X, -pixel_y * spazio_Y, -pixel_y])


    matrice_A = np.array(lista_vettori)
    #viene eseguita una decomposizione SVD (Singular Value Decomposition) sulla matrice matrice_A.
    #Viene estratto il vettore di destra della matrice VT restituita dalla decomposizione SVD,
    U, S, VT_matrix = np.linalg.svd(matrice_A)
    # che corrisponde al più piccolo valore singolare.
    vt = VT_matrix[np.argmin(S)]

    return vt, S, U

![matriceH](matriceH.jpg)

Dopo aver stimato la matrice di proiezione P(H), è possibile calcolare i parametri intrinseci (K) ed estrinseci.

![projMat](projMat.png)

Faremo riferimento solo ad una porzione di questa equazione:

![subMat](subMat.png)

![K](K.jpg)


Calcoliamo i valori della matrice

In [21]:
vt, S, U = sistema_A_P(obj_points, corners)
#Il vettore viene poi rimodellato in una matrice 3x3 per ottenere la matrice di trasformazione H
H = vt.reshape((3, 3))
#La matrice H viene normalizzata dividendo ogni elemento per la norma della matrice
#Viene calcolato il valore dei parametri della matrice K a partire dai valori normalizzati della matrice H
H_norm = H / np.linalg.norm(H)
# calcolo i parametri della matrice K
#vc, che è il prodotto di quattro elementi della matrice H_norm, corrispondenti alle posizioni (1,0), (2,0), (1,1) e (2,1)
vc = H_norm[1,0]*H_norm[2,0]*H_norm[1,1]*H_norm[2,1]
# uc, che è il prodotto di quattro elementi diversi della matrice H_norm
# corrispondenti alle posizioni (0,0), (2,0), (0,1) e (2,1)
uc = H_norm[0,0]*H_norm[2,0]*H_norm[0,1]*H_norm[2,1]
#calcolo alpha, che è la radice quadrata del valore assoluto del prodotto di quattro elementi diversi della matrice H_norm,
#corrispondenti alle posizioni (0,0), (1,0), (0,1) e (1,1), diviso per il prodotto di vc e uc
alpha = np.sqrt(np.abs(H_norm[0,0]*H_norm[1,0]*H_norm[0,1]*H_norm[1,1]/(vc*uc)))
#calcolo beta, che è la radice quadrata del valore assoluto del prodotto di due elementi della matrice H_norm
# corrispondenti alle posizioni (1,1) e (2,1), diviso per il prodotto di vc e alpha alla seconda
beta = np.sqrt(np.abs(H_norm[1,1]*H_norm[2,1]/(vc*alpha**2)))
#calcola gamma, che è il prodotto tra il segno di due elementi diversi della matrice H_norm
#corrispondenti alle posizioni (1,0) e (0,1), e la radice quadrata del valore assoluto della differenza tra alpha alla seconda, beta alla seconda e -1
gamma = np.sign(H_norm[1,0]*H_norm[0,1])*np.sqrt(np.abs(alpha**2*beta**2-1))
#uc dividendo per alpha alla seconda
uc = uc/alpha**2
# vc dividendo per beta alla seconda
vc = vc/beta**2

# costruisci la matrice K
K = np.array([[alpha, gamma, uc],
              [0, beta, vc],
              [0, 0, 1]])
print("la matrice K:\n",K)

la matrice K:
 [[6.03469367e+07 6.34419887e+02 3.33037989e-26]
 [0.00000000e+00 1.05128895e-05 1.60276225e-01]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]




![translation](translation.png)


In [22]:
def getRT(K, H):
    #h1, h2 e h3, che rappresentano rispettivamente la prima, la seconda e la terza colonna della matrice di omografia H
    h1 = H[:,0]
    h2 = H[:,1]
    h3 = H[:,2]
    #lam, che è l'inverso della norma del prodotto tra la matrice di calibrazione K e il vettore h1, normalizzato
    lamb = 1/np.linalg.norm(np.matmul(np.linalg.inv(K),h1))
    #Calcolo il vettore di rotazione r1 moltiplicando l'inversa di K per h1, poi moltiplica per lam
    r1 = lamb * np.matmul(np.linalg.inv(K),h1)
    #Calcolo il vettore di rotazione r2 moltiplicando l'inversa di K per h2, poi moltiplica per lam
    r2 = lamb * np.matmul(np.linalg.inv(K),h2)
    #Calcolo il vettore di rotazione r3 come il prodotto vettoriale tra r1 e r2
    r3 = np.cross(r1,r2)
    #Calcolo il vettore di traslazione T come il prodotto tra lam, l'inversa di K e h3, quindi ne fa la trasposta
    T = np.transpose(lamb *np.matmul(np.linalg.inv(K),h3))
    #Costruisco la matrice di rotazione R come la matrice trasposta dei vettori di rotazione r1, r2 e r3
    R = np.transpose(np.array([r1, r2, r3]))
    return R, T

R, T  = getRT(K, H)
print("rotazione\n",R)
print("traslazione\n",T)

rotazione
 [[ 1.05128760e-05  1.56352102e-05  1.60707490e-08]
 [-1.00000000e+00 -1.48724387e+00  1.68949802e-13]
 [-3.63658142e-08 -7.01555833e-08 -2.64970002e-13]]
traslazione
 [-1.45670983e-04  1.38564351e+01  5.48503240e-07]


In [23]:
img = cv2.imread(path_image + lista_img[20])
def draw_ground(img, R, T, K, dist):
    xg = np.arange(-5, 10, 0.5)
    yg = np.arange(-5, 10, 0.5)
    xx, yy = np.meshgrid(xg, yg)

    dim = xx.shape[0] * xx.shape[1]
    points = np.zeros((dim, 3), np.float32)

    xx = xx.reshape(dim)
    yy = yy.reshape(dim)

    points[:, 0] = xx
    points[:, 1] = yy
    points[:, 2] = np.zeros((dim))

    ground, _ = cv2.projectPoints(points, R, T, K, dist)
    ground = np.squeeze(ground).astype(np.int32)

    img_to_show_res = img.copy()
    for p in ground:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0))

    cv2.imshow("ground", img_to_show_res)
    cv2.waitKey(0)

draw_ground(img, R, T, K, dist)